In [2]:
# Load libraries
import warnings
import numpy as np
import pandas as pd
import os

from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import classification_report
SEED = 90089

In [3]:
# disable deprecation warnings and convergence warning after production 
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

### `LogisticRegression` Classifier on Hyperclygemic patients

In [4]:
# Read data
patient_df = pd.read_csv('../data/hyperglycemic_patients_w_bin_categories.csv', index_col=0)
patient_df = patient_df.drop(['los'], axis=1)

# Previewing data
print("Dataframe shape:   ", patient_df.shape)
print("Dataframe Features:", patient_df.columns)
patient_df.head()

Dataframe shape:    (13243, 26)
Dataframe Features: Index(['glucose_max', 'anchor_age', 'dod', 'gender', 'dbp_mean', 'sbp_mean',
       'glucose_mean', 'heart_rate_mean', 'spo2_mean', 'resp_rate_mean',
       'temperature_mean', 'apsiii', 'glucose_score', 'avg_bmi_value',
       'avg_sofa', 'admission_type', 'label', 'AMBULATORY OBSERVATION',
       'DIRECT EMER.', 'DIRECT OBSERVATION', 'ELECTIVE', 'EU OBSERVATION',
       'EW EMER.', 'OBSERVATION ADMIT', 'SURGICAL SAME DAY ADMISSION',
       'URGENT'],
      dtype='object')


,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,...,label,AMBULATORY OBSERVATION,DIRECT EMER.,DIRECT OBSERVATION,ELECTIVE,EU OBSERVATION,EW EMER.,OBSERVATION ADMIT,SURGICAL SAME DAY ADMISSION,URGENT
0,277.0,36,0,0,64.320000,112.880000,221.272727,101.800000,1.963788,1.239203,...,0,0,0,0,0,0,0,1,0,0
1,259.0,19,1,1,49.300000,74.200000,259.000000,89.750000,1.845098,1.420506,...,0,0,0,0,0,0,1,0,0,0
2,406.0,24,0,0,57.333333,92.848485,292.000000,86.631579,1.983004,1.184393,...,2,0,0,0,0,0,1,0,0,0
3,398.0,31,0,0,78.600000,133.800000,285.000000,100.826087,1.955709,1.400951,...,0,0,0,0,0,0,1,0,0,0
4,487.0,34,0,1,107.062500,164.437500,235.777778,77.000000,1.996731,1.398870,...,2,0,0,0,0,0,0,0,0,1


### LR without `StandardScaler`

In [61]:
# Data split
X = patient_df.loc[:, patient_df.columns != 'label']
y = patient_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
# Tune model with stratifiedKFold cross validation
lr = LogisticRegression(random_state=SEED)

parameters = {
    'penalty' : ['l1', 'l2'],
    'C'       : [0.1, 1, 10],
    'solver'  : ['liblinear', 'saga'], 
    'max_iter': [1000, 2000]}
grid_lr = GridSearchCV(lr, parameters, cv=5, n_jobs=8)
grid_lr.fit(X_train, y_train)

In [31]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_lr.best_estimator_)
print("\n The best parameters across ALL searched params:\n", grid_lr.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 LogisticRegression(C=1, max_iter=1000, penalty='l1', random_state=90089,
                   solver='liblinear')

 The best parameters across ALL searched params:
 {'C': 1, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}


In [63]:
best_lr = LogisticRegression(C=1,
                            penalty='l1',
                            solver='liblinear',
                            random_state=SEED,
                            max_iter=1000,
                            n_jobs=4)

In [64]:
best_lr.fit(X_train, y_train)

LogisticRegression(C=1, max_iter=1000, n_jobs=4, penalty='l1',
                   random_state=90089, solver='liblinear')

In [65]:
print("Model Accuracy based on Testing dataset: {:.5f}".format(best_lr.score(X_test, y_test)))

Model Accuracy based on Testing dataset: 0.52024


### LR Model with `StandardScaler`

In [5]:
numerical_features = ['glucose_max', 'anchor_age', 'dbp_mean', 'sbp_mean',
       'glucose_mean', 'heart_rate_mean', 'spo2_mean', 'resp_rate_mean',
       'temperature_mean', 'apsiii', 'glucose_score', 'avg_sofa']


# Standardized variable for consistent measurement across numerical values
scaler = StandardScaler()
X_std = scaler.fit_transform(patient_df[numerical_features])

# Interpolate categorical variables
X_cat = patient_df[['gender', 'AMBULATORY OBSERVATION',
       'DIRECT EMER.', 'DIRECT OBSERVATION', 'ELECTIVE', 'EU OBSERVATION',
       'EW EMER.', 'OBSERVATION ADMIT', 'SURGICAL SAME DAY ADMISSION',
       'URGENT']]


X = np.concatenate([X_std, X_cat.to_numpy()], axis=1)
y = patient_df["label"]
print("X Standardized data Shape: ", X_std.shape)
print("X Categorical data shape:  ", X_cat.shape)
print("X, y shape:                ", X.shape, y.shape)

X Standardized data Shape:  (13243, 12)
X Categorical data shape:   (13243, 10)
X, y shape:                 (13243, 22) (13243,)


In [6]:
X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [7]:
lr_std = LogisticRegression(random_state=SEED)

parameters = {
    'penalty' : ['l1', 'l2'],
    'C'       : [0.1, 1, 10],
    'solver'  : ['liblinear', 'saga'], 
    'max_iter': [100, 500]}
grid_lr_std = GridSearchCV(lr_std, parameters, cv=5, n_jobs=6)
grid_lr_std.fit(X_train_std, y_train_std)

d:\miniconda3\envs\cv\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=90089), n_jobs=6,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [100, 500],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [20]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_lr_std.best_estimator_)
print("\n The best parameters across ALL searched params:\n", grid_lr_std.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 LogisticRegression(C=1, penalty='l1', random_state=90089, solver='saga')

 The best parameters across ALL searched params:
 {'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}


In [21]:
best_lr_std = LogisticRegression(C = 1,
                                penalty = 'l1',
                                solver = 'saga',
                                random_state=SEED,
                                max_iter=100, 
                                n_jobs=8)
best_lr_std.fit(X_train_std, y_train_std)

d:\miniconda3\envs\cv\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(C=1, n_jobs=8, penalty='l1', random_state=90089,
                   solver='saga')

In [22]:
print("Model Accuracy based on Testing dataset: {:.5f}".format(best_lr_std.score(X_test_std, y_test_std)))

Model Accuracy based on Testing dataset: 0.64439


In [8]:
y_pred = best_lr_std.predict(X_test_std)
print(classification_report(y_test_std, y_pred))

NameError: name 'best_lr_std' is not defined